Scraping the 1000 links

In [ ]:
import requests
from bs4 import BeautifulSoup
 #---------------------------------------- URL ---------------------------------------------
movie_links1000 = [] # creating a list to get all the links

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
for i in range(1,2):
    fhand = requests.get(f'https://www.metacritic.com/browse/movie/?releaseYearMin=1910&releaseYearMax=2023&page={i}', headers = headers)

    soup = BeautifulSoup(fhand.content, "html.parser") # Using BeatifulSoup to extract the data from the fhand
    hyperlink_list = soup.find_all('a', class_='c-finderProductCard_container g-color-gray80 u-grid') #Telling BeatifulSoup what to extract.
    for link in hyperlink_list:
        movie_page =  'https://www.metacritic.com' + link['href'] # Appending the links to the common url.
        movie_links1000.append(movie_page) #Adding the link from each movie to the list that we are going to use to find out data.
print(hyperlink_list)

Scraping all the information from the movies

In [8]:
# ----------------------------------- MOVIE NAME -------------------------------------------
movie_dict = {} # Creating a dictionary to store all the data requested.
for movie_url in movie_links1000:
    movie_page = requests.get(movie_url, headers = headers)
    movie_page_soup = BeautifulSoup(movie_page.content, "html.parser") # Using BeatifulSoup to extract the data from the movie_page.
    movie_name_list = movie_page_soup.find_all('div', class_ = "c-productHero_title g-inner-spacing-bottom-medium g-outer-spacing-top-medium")
    for i in movie_name_list: #iterating over the movie_names to get the clean name.
        movie_name_clean = i.get_text().strip()
    view_all = movie_page_soup.find_all('a', class_ = "c-globalHeader_viewMore g-color-gray50 g-text-xxsmall u-text-uppercase")
    for i in view_all:
        view_all_link = 'https://www.metacritic.com' + i['href'] #extracting the url to the "view all" page
        #print(view_all_link)
    view_all_page = requests.get(view_all_link, headers = headers) #using bs4 to scrape the "view all" page
    view_all_soup = BeautifulSoup(view_all_page.content, "html.parser")

    credit_tables_list = view_all_soup.find_all('div', class_ = "c-productCredits g-outer-spacing-bottom-xlarge")
# ----------------------------------- DIRECTORS ---------------------------------------------
    directors_list = [] #Here we will store our director names.
    for i in credit_tables_list:
        if "Directed By" in i.get_text().strip():
            director_names = i.find_all('a')
            for j in director_names:
                director_name = j.get_text().strip() #cleaning our data.
                directors_list.append(director_name) #appending the names to the list
                director_t = tuple(directors_list) # Converting the list to a tuple so it can be used as key.
# ---------------------------------- CAST ---------------------------------------------------
    casts = []
    cast_table = view_all_soup.find_all('div', class_="c-productCredits g-outer-spacing-bottom-xlarge")
    for i in cast_table:
        t_title = i.find('h3')
        if t_title.get_text().strip() == 'Cast': #Creating a condition so it only iterates over 'Cast'.
            cast_names = i.find_all('a')
            for j in cast_names:
                cast_name = j.get_text().strip() #cleaning the data.
                casts.append(cast_name) #appendig to our future subvalue 'casts'.
            #print(casts)
# --------------------------- DICT ----------------------------------------------------------
    dir_cast = {'Directors' : director_t, 'Cast': casts}
    movie_dict[movie_name_clean] = dir_cast #Creating the dictionary with all the information needed.


movie_dict

{'Dekalog (1988)': {'Directors': ('Krzysztof Kieslowski',),
  'Cast': ['Artur Barcis',
   'Olgierd Lukaszewicz',
   'Olaf Lubaszenko',
   'Aleksander Bardini',
   'Krystyna Janda',
   'Piotr Machalica',
   'Jan Tesarz',
   'Stanislaw Gawlik',
   'Krzysztof Kumor',
   'Katarzyna Piwowarczyk',
   'Maciej Szary',
   'Henryk Baranowski',
   'Jerzy Stuhr',
   'Daniel Olbrychski',
   'Grazyna Szapolowska',
   'Adrianna Biedrzynska',
   'Miroslaw Baka',
   'Anna Polony',
   'Ewa Blaszczyk',
   'Maria Koscialkowska',
   'Zbigniew Zamachowski',
   'Janusz Gajos',
   'Wojciech Klata',
   'Maria Pakulnis',
   'Krzysztof Globisz',
   'Teresa Marczewska',
   'Maja Barelkowska',
   'Wladyslaw Kowalski',
   'Maja Komorowska',
   'Henryk Bista',
   'Joanna Szczepkowska',
   'Stefania Iwinska',
   'Boguslaw Linda',
   'Tadeusz Lomnicki',
   'Jan Jankowski',
   'Maciej Stuhr',
   'Marian Opania',
   'Jolanta Pietek-Górecka',
   'Adam Hanuszkiewicz',
   'Agnieszka Brustman',
   'Krystyna Drochocka',
   '

Creating the CSV

In [ ]:
import csv

csv_file_path = '[Menacho_Caffarena]_movies.csv.csv' # Specifyng the file name

with open(csv_file_path, 'w', newline='') as csv_file:  #Creating the file with the requested name
    csv_writer = csv.writer(csv_file)

    # Writing the headers
    csv_writer.writerow(['movie_name_clean', 'director_t', 'casts'])

    # Writing the data rows
    for movie_name, info_dict in movie_dict.items():
        director_t = ', '.join(info_dict['Directors'])
        casts = ', '.join(info_dict['Cast'])
        csv_writer.writerow([movie_name, director_t, casts])


Asking the user for prompts.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
'''
Using the library CountVectorizer vectorize our data in order to get a matrix of data points.
Then we used the cosine_similary function to get the cosine_similary of the authors.
'''
choice = input('What do you want to check on Metacrtitics? Please choose ‘movie’, ‘people’ or ‘comparison’ ')
    #--------------------------------------------CHOICE 1-------------------------------------------------------------
if choice.lower() == 'movie':
    choice2 = input('Please pick a movie name here: ')
    found = False  # Flag to check if the movie is found
    for movie_name_clean, dir_cast in movie_dict.items(): #Searching for the items requested form the user.
        if movie_name_clean.lower().strip() == choice2.lower().strip(): #comaparing if the director name requested matches any director in our dictionary.
            found = True
            for key, value in dir_cast.items(): #Looking for the items inside the subdictionary.
                if key == 'Directors':
                    print('The director is', ','.join(value))
                elif key == 'Cast':
                    print('The cast members are:', ','.join(value))
    if not found:
        print(f"Movie '{choice2}' not found in the dictionary.")
#--------------------------------------------CHOICE 2 -------------------------------------------------------------
elif choice.lower() == 'people':
    choice2 = input('Pick director please ')
    movie_list = [] #Creating a list to append the movie names from the actor requested.
    cast_list = []  #Creating a list to append the casts from the actor requested.
    director_found = False
    #--------------block1-------------
    for key, value in movie_dict.items(): #Iterating over the dictionary to see if the director is there.
        directors_lower = []
        for director in value['Directors']:
            directors_lower.append(director.strip().lower()) #Appending the director without the format.
        if choice2.lower().strip() in directors_lower:
            director_found = True
            movie_list.append(key)
            cast_list.extend(value['Cast'])

    #----------block2 starts--------
    if director_found:
        cast_times = {} #Creating a dictionary to append the cast members names and the number of times they work with that director.
        for actor in cast_list:
            cast_times[actor] = cast_times.get(actor, 0) + 1

        print(f'The director {choice2} has directed {", ".join(movie_list)}. Also, the director worked with: ')
        for actor, times_worked in cast_times.items():
            print(f'{actor} {times_worked} ', end=',')
    else:
        print(f"Director '{choice2}' not found in the dictionary.")

#if the user wants to compare, we ask for the two actors.
elif choice.lower() == 'comparison':
    choice1 = input('pick 1 director ')
    choice2 = input('pick another director ')

    movie_list1 = [] #Creating empty lists to store the cast and movies where the author has worked.
    cast_list1 = []
    for key, value in movie_dict.items(): #Checking if the author selected is in our dictionary.
        directors_lower1 = []
        for director in value['Directors']:
            directors_lower1.append(director.strip().lower())
        if choice1.lower().strip() in directors_lower1: #cleaning the choice from the user
            movie_list1.append(key) #Appending the movie name of the movies where the dictector worked.
            cast_list1.extend(value['Cast']) #Extending the cast of the movies that the director has worked with.
    cast_times1 = {}
    for actor in cast_list1:
        cast_times1[actor] = cast_times1.get(actor, 0) + 1 #Creating a dictionary with the cast and the number of times the author has worked with each member.

    #Repeating the same steps for director 2.
    movie_list2 = []
    cast_list2 = []
    for key, value in movie_dict.items():
        directors_lower2 = []
        for director in value['Directors']:
            directors_lower2.append(director.strip().lower())
        if choice2.lower().strip() in directors_lower2:
            movie_list2.append(key)
            cast_list2.extend(value['Cast'])

    combined_cast = [str(cast_list1), str(cast_list2)]

    # Vectorizing and calculating the cosine similarity
    vectorizer = CountVectorizer().fit_transform(combined_cast)
    cosine_sim = cosine_similarity(vectorizer)


    cast_times2 = {}
    for actor in cast_list2:
        cast_times2[actor] = cast_times2.get(actor, 0) + 1

    print(f"{choice1} has worked with {str(cast_times1)} and {choice2} has worked with {str(cast_times2)}.")
    print(f"The Cosine Similarity between {choice1} and {choice2} is {cosine_sim[0][1]:.2f}")


else:
    print('Please select either movie, people or comparison')

What do you want to check on Metacrtitics? Please choose ‘movie’, ‘people’ or ‘comparison’ movie
Please pick a movie name here: Tokyo Story
The director is Yasujirô Ozu
The cast members are: Chishû Ryû,Chieko Higashiyama,Setsuko Hara,Haruko Sugimura,Sô Yamamura,Kuniko Miyake,Kyôko Kagawa,Eijirô Tôno,Nobuo Nakamura,Shirô Ôsaka,Hisao Toake,Teruko Nagaoka,Mutsuko Sakura,Toyo Takahashi,Tôru Abe,Sachiko Mitani,Zen Murase,Mitsuhiro Môri,Junko Anan,Ryôko Mizuki,Yoshiko Togawa,Kazuhiro Itokawa,Fumio Tooyama,Keijirô Morozumi,Tsutomu Niijima,Shôzô Suzuki,Yoshiko Tashiro,Haruko Chichibu,Takashi Miki,Binnosuke Nagao
